In [1]:
import pandas as pd
import spacy
import wikipedia

import entitylinker
from sklearn.feature_extraction.text import TfidfVectorizer



/Users/hiddemakimei/Desktop/git/vu/web/kb/wdps_group27/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
llama_new_context_with_model: n_ctx_per_seq (512) < n_ctx_train (4096) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_f32_f32                    (not 

In [ ]:
!!python -m spacy download en_core_web_sm


In [2]:
df = pd.read_csv('/Users/hiddemakimei/Desktop/git/vu/web/kb/data/train.csv')

In [3]:
df.head()

,claim,explanation,label,golden entity,mention,result entity,discard number
0,The Hit is the first film Stephen Frears direc...,No information shows that The Hit is the first...,NOT ENOUGH INFO,"['The_Hit_(1984_film)', 'Tim_Roth']",['Tim Roth'],NaN,NaN
1,Gemini is an Indian Tamil-language crime actio...,Gemini is an Indian Tamil-language crime actio...,SUPPORT,"['Gemini_(2002_Tamil_film)', 'AVM_Productions']",['AVM Productions'],NaN,NaN
2,Zach Galifianakis is an American actor who app...,Zach Galifianakis is an American actor who app...,REFUTE,"['Zach_Galifianakis', 'Due_Date']",['Due Date'],NaN,NaN
3,Mark Alan Ruffalo went on to star in a 2002 Am...,Mark Alan Ruffalo went on to star in Shutter I...,REFUTE,"['Mark_Ruffalo', 'Shutter_Island_(film)', 'Shu...","['Shutter Island', '2003 novel of the same name']",NaN,NaN
4,Martin is currently writing the sixth novel in...,"Martin is currently writing the sixth novel, T...",SUPPORT,"['A_Song_of_Ice_and_Fire', 'The_Winds_of_Winter']",['The Winds of Winter'],NaN,NaN


In [4]:
df_check = df[df['result entity'].notna()]

In [5]:
df_check.head()

,claim,explanation,label,golden entity,mention,result entity,discard number
5,Glenn Lewis Frey sang the lead vocals on the t...,Peaceful Easy Feeling was the third single by ...,NOT ENOUGH INFO,"['Glenn_Frey', 'Peaceful_Easy_Feeling', 'Berni...","['Peaceful Easy Feeling', 'Bernie Leadon']","['Peaceful_Easy_Feeling', 'Bernie_Leadon']",1.0
6,Glenn Lewis Frey sang the lead vocals on the t...,Glenn Lewis Frey sang the lead vocals on the s...,SUPPORT,"['Glenn_Frey', 'Peaceful_Easy_Feeling', 'Berni...","['Peaceful Easy Feeling', 'Bernie Leadon']","['Peaceful_Easy_Feeling', 'Bernie_Leadon']",1.0
7,Alison Brie is an American actress who has won...,Alison Brie is an American actress who has won...,NOT ENOUGH INFO,"['Alison_Brie', 'Screen_Actors_Guild', 'Trade_...","['Screen Actors Guild Award', 'labor union']","['Alison_Brie', 'Trade_union']",2.0
8,Alison Brie is an American actress who has won...,Alison Brie is an American actress who has won...,SUPPORT,"['Alison_Brie', 'Screen_Actors_Guild', 'Trade_...","['Screen Actors Guild Award', 'labor union']","['Alison_Brie', 'Trade_union']",2.0
25,L.A. Law is an American legal drama television...,Homophobia encompasses a range of negative att...,NOT ENOUGH INFO,"['L.A._Law', 'Homophobia', 'Bisexual']","['homophobia', 'bisexual']","['Homophobia', 'Bisexual']",1.0


In [6]:


nlp = spacy.load("en_core_web_sm")
claim = "Bohemian Rhapsody is a song by the British rock band Queen for their 1975 album that was released on 21 November 1975."
doc = nlp(claim)

# Extract Named Entities
entities = [(ent.text, ent.label_) for ent in doc.ents]
print("Named Entities:", entities)

entities_text =[(ent.text) for ent in doc.ents]


Named Entities: [('Bohemian Rhapsody', 'NORP'), ('British', 'NORP'), ('Queen', 'PERSON'), ('1975', 'DATE'), ('21 November 1975', 'DATE')]


In [7]:

temp_entities = entities_text

page_list = []

while len(temp_entities) != 0 :

    try:
        print(f"Searching for: {' '.join(temp_entities)}")
        page = wikipedia.page(' '.join(temp_entities))
        print(f"Title: {page.title}")
        page_list.append(page.content)

    except Exception as e:
        print(f"Error finding page for {temp_entities[-1]}: {e}")


    temp_entities = temp_entities[:-1]




## to do: the right combination of entities


Searching for: Bohemian Rhapsody British Queen 1975 21 November 1975
Title: Bohemian Rhapsody
Searching for: Bohemian Rhapsody British Queen 1975
Title: Bohemian Rhapsody
Searching for: Bohemian Rhapsody British Queen
Title: Bohemian Rhapsody
Searching for: Bohemian Rhapsody British
Title: Bohemian Rhapsody
Searching for: Bohemian Rhapsody
Title: Bohemian Rhapsody


In [8]:
from transformers import pipeline
pipe = pipeline("text-classification", model="Dzeniks/roberta-fact-check")


pipe("test")


/Users/hiddemakimei/Desktop/git/vu/web/kb/wdps_group27/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/hiddemakimei/Desktop/git/vu/web/kb/wdps_group27/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Device set to use cpu


[{'label': 'LABEL_0', 'score': 0.6727511286735535}]

In [9]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification

tokenizer = RobertaTokenizer.from_pretrained('Dzeniks/roberta-fact-check')
model = RobertaForSequenceClassification.from_pretrained('Dzeniks/roberta-fact-check')

# Define the claim with evidence to classify
evidence = """
Justine Tanya Bateman (born February 19, 1966) is an American writer, producer, and actress . She is best known for her regular role as Mallory Keaton on the sitcom Family Ties (1982 -- 1989). Until recently, Bateman ran a production and consulting company, SECTION 5 . In the fall of 2012, she started studying computer science at UCLA.
"""

claim = 'Justine Bateman is a swimmer.'

# Tokenize the claim with evidence
x = tokenizer.encode_plus(claim, evidence, return_tensors="pt")

model.eval()
with torch.no_grad():
  prediction = model(**x)

print(prediction)

print(torch.argmax(prediction[0]).item())


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


SequenceClassifierOutput(loss=None, logits=tensor([[-0.9279,  1.4750]]), hidden_states=None, attentions=None)
1


/Users/hiddemakimei/Desktop/git/vu/web/kb/wdps_group27/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
!!pip install sentencepiece


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

documents = page_list  # Content of linked Wikipedia pages
tfidf = TfidfVectorizer().fit_transform(documents)
similarity_matrix = (tfidf * tfidf.T).toarray()
print(similarity_matrix)

[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]


In [13]:
from transformers import T5ForConditionalGeneration, T5Tokenizer



claim = "Bohemian Rhapsody is a song by the British rock band Queen for their 1975 album that was released on 21 November 1975."

# def generate_golden_explanation_with_t5(claim, evidence):
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')

input_text = f"claim: {claim} evidence: {page_list[0]:200}"
input_ids = tokenizer.encode(input_text, return_tensors="pt")

output_ids = model.generate(input_ids, max_length=200)
explanation = tokenizer.decode(output_ids[0], skip_special_tokens=True)
explanation

Token indices sequence length is longer than the specified maximum sequence length for this model (12625 > 512). Running this sequence through the model will result in indexing errors


'the song was a\'stupid\' and a\'stupid\' song. the band\'s first single was "Bohemian Rhapsody" in 1975. the band\'s first single was "Bohemian Rhapsody" in the 1980s. the band\'s first single was "Bohemian Rhapsody" in the 1980s.'